# Padding in conv layer

padding是开始接触conv层最麻烦的一个参数。不同的padding对conv层的输出的大小有影响。跟strides，filter size，filter个数混到一起，好几个变量才能计算出输出层的大小，看上去挺复杂的。

之所以它显得很麻烦，有一个原因是：一般的教程介绍经典网络都是从AlexNet开始的，而它用了11x11, 5x5, 3x3不同大小的卷积核，也使用了不同的padding size为了能正确的理解它需要仔细的计算这些数字。并且cs231n的说法，alexnet原文的描述中第一层的数字是有问题的[1]。这更加迷惑了初学者。

但到了vgg，就变得很简单了。因为它只使用3x3的卷积核，padding为1，这样就保持了conv层的输入和输出的空间大小不变。空间大小的变化只发生在pooling层，并且都是长宽同时减半。

包括后面的其他网络，例如ResNet，虽然使用了不同大小的卷积核，但都保持了输入的空间大小，至于为了保持空间的大小，需要几个padding，用公式很容易计算出来，就显得不那么重要了。

cs231n里对于padding相关的计算有一个很实用的公式，来计算输出的大小：

    (W−F+2P)/S+1
 
http://cs231n.github.io/convolutional-networks/

W：输入的宽度
F：卷积核的大小
P：填充的大小，之所以是2P是因为两边都填充
S：步长的大小

这个公式从形象上也不难理解，想象一个小的边长为F的正方形（kernel）在一个大的宽度为W的矩形（输入图片）上从左（正方形的左边对齐矩形的左边）到右（正方形的右边对齐矩形的右边）滑动。每次滑动时向右移动S个像素。在每一个固定的位置，小正方形会产生“输出图片”的一个像素。滑动到下一个位置会产生另一个像素。


[1]: Real-world example. The Krizhevsky et al. architecture that won the ImageNet challenge in 2012 accepted images of size [227x227x3]. On the first Convolutional Layer, it used neurons with receptive field size F=11F=11, stride S=4S=4 and no zero padding P=0P=0. Since (227 - 11)/4 + 1 = 55, and since the Conv layer had a depth of K=96K=96, the Conv layer output volume had size [55x55x96]. Each of the 55*55*96 neurons in this volume was connected to a region of size [11x11x3] in the input volume. Moreover, all 96 neurons in each depth column are connected to the same [11x11x3] region of the input, but of course with different weights. As a fun aside, if you read the actual paper it claims that the input images were 224x224, which is surely incorrect because (224 - 11)/4 + 1 is quite clearly not an integer. This has confused many people in the history of ConvNets and little is known about what happened. My own best guess is that Alex used zero-padding of 3 extra pixels that he does not mention in the paper.


In [7]:
import tensorflow as tf

image = tf.placeholder(tf.float32, shape=[1, 227, 227, 3])
filter_ = tf.Variable(tf.truncated_normal([11, 11, 3, 96]))   
conv = tf.nn.conv2d(image, filter_, strides=[1, 4, 4, 1], padding='VALID')

print 'image =', image
print 'filter =', filter_
print 'conv =', conv

image = Tensor("Placeholder_5:0", shape=(1, 227, 227, 3), dtype=float32)
filter = <tf.Variable 'Variable_3:0' shape=(11, 11, 3, 96) dtype=float32_ref>
conv = Tensor("Conv2D_3:0", shape=(1, 55, 55, 96), dtype=float32)


In [10]:

image = tf.placeholder(tf.float32, shape=[1, 224, 224, 3])
paddings = [[0, 0], [1, 2], [1, 2], [0, 0]]
padded_image = tf.pad(image, paddings, "CONSTANT")
print 'padded image =', padded_image

filter_ = tf.Variable(tf.truncated_normal([11, 11, 3, 96]))
conv = tf.nn.conv2d(padded_image, filter_, strides=[1, 4, 4, 1], padding='VALID')
print 'conv =', conv


padded image = Tensor("Pad_1:0", shape=(1, 227, 227, 3), dtype=float32)
conv = Tensor("Conv2D_6:0", shape=(1, 55, 55, 96), dtype=float32)
